In [1]:
# gerekli kütüphaneler yükleniyor
from surprise import Dataset, KNNBasic, accuracy  # surprise: öneri sistemleri için kullanılır
from surprise.model_selection import train_test_split  # eğitim ve test verisi için bölme fonksiyonu


In [2]:
# MovieLens 100K veri seti yükleniyor
# veri seti: (kullanıcı id, film id, kullanıcı puanı)
data = Dataset.load_builtin("ml-100k")  # 100.000 kullanıcı-ürün-puan verisi içerir



In [3]:
# veri seti eğitim ve test olmak üzere ikiye bölünüyor (%20 test)
trainset, testset = train_test_split(data, test_size=0.2)

In [4]:
# kullanılacak benzerlik metriklerini tanımlıyoruz
model_options = {
    "name": "cosine",         # kullanıcılar/filmler arası benzerlik için kosinüs benzerliği
    "user_based": True        # True => kullanıcılar arası benzerlik yapılacak (False => ürünler arası)
}


In [5]:
# KNN tabanlı bir model oluşturuluyor
model = KNNBasic(sim_options=model_options)  # KNNBasic: Basit K-En Yakın Komşu algoritması

# model eğitim verisi ile eğitiliyor
model.fit(trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [6]:
# test verisi ile tahmin yap
predictions = model.test(testset)

In [7]:
# Tahminlerin doğruluğunu RMSE (Root Mean Squared Error) metriğiyle hesaplayalım
rmse = accuracy.rmse(predictions)
print(f"Test RMSE: {rmse:.4f}")

RMSE: 1.0247
Test RMSE: 1.0247


In [8]:
def get_top_n(predictions, n=10):
    # tahmin edilen puanlardan her kullanıcıya ait en yüksek n tanesini döndürür
    top_n = {}  # kullanıcıya göre organize edilen öneri listesi

    for uid, iid, true_r, est, _ in predictions:
        # eger kullanıcı daha önce eklenmemişse, kullanıcı için boş liste oluştur
        if not top_n.get(uid):
            top_n[uid] = []
        
        # kullanıcıya ait tahmin edilen ürün ve skorları ekle
        top_n[uid].append((iid, est))

    # her kullanıcı için puan sırasına göre büyükten küçüğe sırala ve en iyi n tanesini al
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)  # tahmin edilen puana göre sırala
        top_n[uid] = user_ratings[:n]

    return top_n  # kullanıcıya göre n öneri listesi döndürülür


In [9]:
# kaç öneri yapılacağı belirleniyor
n = 5

# tüm kullanıcılar için en iyi 5 öneri alınır
top_n = get_top_n(predictions, n)

# belirli bir kullanıcıya önerilen ürünler yazdırılıyor
user_id = "2"  # string türünde olmalı, çünkü MovieLens veri setindeki kullanıcı id’leri string
print(f"top {n} recommendation for user {user_id}")

# kullanıcıya önerilen her ürünü ve puanını yazdır
for item_id, rating in top_n[user_id]:
    print(f"item id: {item_id}, score: {rating}")


top 5 recommendation for user 2
item id: 127, score: 4.350154044349388
item id: 242, score: 4.049253252856681
item id: 293, score: 3.9750639986584586
item id: 287, score: 3.800509680448711
item id: 276, score: 3.724925249542078
